In [52]:
import numpy as np
import pickle
import re
import os
import random

In [46]:
data_folder = "../ds19/"
traces = {id: [None] * 100 for id in range(100)}
indices = {id: 0 for id in range(100)}
patterns = [re.compile("^\d{2}-\d{2}.cell"), re.compile("^\d{2}-\d{1}.cell"), re.compile("^\d{1}-\d{2}.cell"), re.compile("^\d{1}-\d{1}.cell")]

In [47]:
def process(filename, trace_id):
    with open(data_folder + filename) as f:
        raw_file = np.loadtxt(f)
        
    converted_file = np.asarray([row[0] * row[1] for row in raw_file])
    
    traces[trace_id][indices[trace_id]] = converted_file
    indices[trace_id] += 1

In [48]:
for filename in os.listdir(data_folder):
    
    # 2-digit, 2-digit
    if patterns[0].match(filename):
        process(filename, int(filename[:2]))
    # 2-digit, 1-digit
    elif patterns[1].match(filename):
        process(filename, int(filename[:2]))
    # 1-digit, 2-digit or 1-digit, 1-digit
    elif patterns[2].match(filename) or patterns[3].match(filename):
        process(filename, int(filename[0]))

0


In [49]:
print(indices)

{0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100, 6: 100, 7: 100, 8: 100, 9: 100, 10: 100, 11: 100, 12: 100, 13: 100, 14: 100, 15: 100, 16: 100, 17: 100, 18: 100, 19: 100, 20: 100, 21: 100, 22: 100, 23: 100, 24: 100, 25: 100, 26: 100, 27: 100, 28: 100, 29: 100, 30: 100, 31: 100, 32: 100, 33: 100, 34: 100, 35: 100, 36: 100, 37: 100, 38: 100, 39: 100, 40: 100, 41: 100, 42: 100, 43: 100, 44: 100, 45: 100, 46: 100, 47: 100, 48: 100, 49: 100, 50: 100, 51: 100, 52: 100, 53: 100, 54: 100, 55: 100, 56: 100, 57: 100, 58: 100, 59: 100, 60: 100, 61: 100, 62: 100, 63: 100, 64: 100, 65: 100, 66: 100, 67: 100, 68: 100, 69: 100, 70: 100, 71: 100, 72: 100, 73: 100, 74: 100, 75: 100, 76: 100, 77: 100, 78: 100, 79: 100, 80: 100, 81: 100, 82: 100, 83: 100, 84: 100, 85: 100, 86: 100, 87: 100, 88: 100, 89: 100, 90: 100, 91: 100, 92: 100, 93: 100, 94: 100, 95: 100, 96: 100, 97: 100, 98: 100, 99: 100}


In [50]:
with open('../ds19.npy', 'wb') as f:
    pickle.dump(traces, f)

In [53]:
shapelets = [None] * 100

for i in range(100):
    shapelets[i] = random.choice(traces[i])

In [57]:
with open('../results/shapelets/num=0size=0', 'wb') as f:
    pickle.dump(shapelets, f)

In [56]:
print(len(shapelets))
print(shapelets[66].shape)

100
(3016,)
